In [1]:
from jax import random
import jax
import neural_tangents as nt
from neural_tangents import stax

2022-11-16 18:41:19.412211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(512), stax.Relu(),
    stax.Dense(512), stax.Relu(),
    stax.Dense(1)
)

In [3]:
n_train = 5000
n_test = 1
d = 100 
key1, key2, key3 = random.split(random.PRNGKey(1), num=3)
output_shape, params = init_fn(key1, input_shape=(d,))
param_count = sum(x.size for x in jax.tree_leaves(params))
delta = jax.numpy.sqrt(param_count/n_train)
cov = jax.numpy.identity(d, dtype=None)
cov = cov.at[0,0].set(1+delta)
mean = jax.numpy.zeros(d)

/tmp/ipykernel_1074742/4235229312.py:6: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  param_count = sum(x.size for x in jax.tree_leaves(params))


In [4]:
x_train = jax.random.multivariate_normal(key2, mean, cov, (n_train,))
y_train = random.uniform(key2, shape=(n_train, 1))  # training targets
x_test = jax.random.multivariate_normal(key3, mean, cov, (n_test,))

In [5]:
def returnsNTkKernel(x_train, kernel_fn, x=None, train=True):
    if train:
        ntk_train_train = kernel_fn(x_train, x_train, 'ntk')
        return ntk_train_train
    ntk_test_train = kernel_fn(x, x_train, 'ntk')
    return ntk_test_train

def EigenDecompositonOfKernel(ntk_train_train):
    eigenvalues, eigenvectors = jax.numpy.linalg.eigh(ntk_train_train)
    return eigenvalues, eigenvectors

def feature(y_train, ntk_test_train, eigenvalues, eigenvectors, i):
    lambda_i, v_i = eigenvalues[i], eigenvectors[i].reshape(eigenvectors[i].shape[0],1)
    output = ntk_test_train @ v_i @ v_i.T @ y_train
    output = output/lambda_i
    return output

def returnsXPerturbed(x_test, epsilon, key):
    noise = random.normal(key, x_test.shape)
    noise_norm = jax.numpy.linalg.norm(noise, axis=1)
    noise_norm = noise_norm.reshape(noise_norm.shape[0],1)
    normed_noise = epsilon*noise/noise_norm
    x_perturbed = x_test+normed_noise
    return x_perturbed

In [6]:
ntk_train_train = returnsNTkKernel(x_train, kernel_fn, x=None, train=True)
eigenvalues, eigenvectors = EigenDecompositonOfKernel(ntk_train_train)

In [ ]:
norms={}
for epsilon in [0.5, 0.7, 1, 2]:
    norms[str(epsilon)] = []
    x_perturbed = returnsXPerturbed(x_test, epsilon, key3)
    ntk_test_train_perturbed = returnsNTkKernel(x_train, kernel_fn, x_perturbed, train=False)
    ntk_test_train_clean = returnsNTkKernel(x_train, kernel_fn, x_test, train=False)
    for i in range(x_train.shape[0]):
        f_x_delta = feature(y_train, ntk_test_train_perturbed, eigenvalues, eigenvectors, i)
        f_x = feature(y_train, ntk_test_train_clean, eigenvalues, eigenvectors, i)
        difference = jax.numpy.linalg.norm(f_x_delta-f_x)
        norms[str(epsilon)].append(difference)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
fig = plt.figure(figsize=(15,5))
gs = matplotlib.gridspec.GridSpec(1, 2, width_ratios=[2, 2]) 
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
for epsilon in norms.keys():
    if epsilon in ['0.5']:
        ax1.plot(eigenvalues, norms[epsilon], label=epsilon)
        ax2.plot(eigenvalues, label=epsilon)
ax1.set_ylabel("||f_x_perturbed-f_x||")
ax1.set_xlabel("Eigenvalues")
ax2.set_ylabel("Eigenvalues")
plt.legend()
plt.show()